daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
from spx_data_update import UpdateSP500Data
import numpy as np
import pandas as pd
import nest_asyncio
from time import time
import plistlib
file_name = UpdateSP500Data.DATA_BASE_PATH / 'config.plist'
f = open(str(file_name), 'rb')
pl = plistlib.load(f)
illiquid_dict = pl['illiquid_equity']
illiquid_equity = sum(illiquid_dict.values()) * 0.5
illiquid_equity

531500.0

In [2]:
before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [12]:
z_score = np.array([-0.5, -1, -1.5])
leverage = np.array(range(1, 3, 1))
num_expiries = 3

after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 268.8
elapsed: 280.8
2733.14
19.31
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2768922.6227', currency='BASE', modelCode='')]


In [13]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [14]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-0.5,-1.0,-1.5
2018.12.21,2675,2640,2620
2019.01.18,2620,2555,2505
2019.02.15,2565,2470,2400


,-0.5,-1.0,-1.5
2018.12.21,23.75,16.80,13.85
2019.01.18,29.90,20.05,14.85
2019.02.15,35.40,22.50,16.15


,-0.5,-1.0,-1.5
2018.12.21,23.2/24.3,16.3/17.3,13.5/14.2
2019.01.18,29.5/30.3,19.6/20.5,14.4/15.3
2019.02.15,35.0/35.8,22.0/23.0,15.9/16.4


In [15]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-0.5,-1.0,-1.5
2018.12.21,-2.13%,-3.41%,-4.14%
2019.01.18,-4.14%,-6.52%,-8.35%
2019.02.15,-6.15%,-9.63%,-12.19%


In [16]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-0.5,-1.0,-1.5
2018.12.21,"$28,500","$21,840","$18,005"
2019.01.18,"$38,870","$26,065","$19,305"
2019.02.15,"$46,020","$31,500","$22,610"


,-0.5,-1.0,-1.5
2018.12.21,"$59,375","$42,000","$34,625"
2019.01.18,"$77,740","$52,130","$40,095"
2019.02.15,"$92,040","$60,750","$45,220"


In [17]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-0.5,-1.0,-1.5
2018.12.21,0.87%,0.61%,0.51%
2019.01.18,1.09%,0.73%,0.54%
2019.02.15,1.30%,0.82%,0.59%


,-0.5,-1.0,-1.5
2018.12.21,1.74%,1.23%,1.01%
2019.01.18,2.19%,1.47%,1.09%
2019.02.15,2.59%,1.65%,1.18%


In [18]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,12,13,13
2019.01.18,13,13,13
2019.02.15,13,14,14


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,25,25,25
2019.01.18,26,26,27
2019.02.15,26,27,28


In [19]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-0.5,-1.0,-1.5
2018.12.21,"$614,686","$611,374","$581,539"
2019.01.18,"$602,404","$505,099","$433,339"
2019.02.15,"$538,054","$428,383","$358,610"


,-0.5,-1.0,-1.5
2018.12.21,"$1,280,595","$1,175,720","$1,118,345"
2019.01.18,"$1,204,809","$1,010,199","$900,013"
2019.02.15,"$1,076,109","$826,168","$717,220"


In [20]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$1,280,595","$1,175,720","$1,118,345"
2019.01.18,"$1,204,809","$1,010,199","$900,013"
2019.02.15,"$1,076,109","$826,168","$717,220"


In [21]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,"$3,210,000","$3,432,000","$3,406,000"
2019.01.18,"$3,406,000","$3,321,500","$3,256,500"
2019.02.15,"$3,334,500","$3,458,000","$3,360,000"


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$6,687,500","$6,600,000","$6,550,000"
2019.01.18,"$6,812,000","$6,643,000","$6,763,500"
2019.02.15,"$6,669,000","$6,669,000","$6,720,000"


In [ ]:
capital_at_risk